In [1]:
import numpy as np
import pandas as pd 
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [2]:
testset = pd.read_csv("Desktop/test.csv")
trainingset = pd.read_csv("Desktop/train.csv") 
test  = testset.copy()
train  = trainingset.copy()

In [3]:
test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.24,Ideal,G,VVS1,62.1,56.0,3.97,4.00,2.47
1,1,1.21,Very Good,F,VS2,62.9,54.0,6.78,6.82,4.28
2,2,0.50,Fair,E,SI1,61.7,68.0,5.09,5.03,3.12
3,3,0.50,Ideal,D,SI2,62.8,56.0,5.06,5.03,3.17
4,4,1.55,Ideal,E,SI2,62.3,55.0,7.44,7.37,4.61


In [4]:
train.head()

,id,carat,cut,color,clarity,depth,table,price,x,y,z
0,0,1.01,Very Good,E,SI2,60.0,60.0,4540,6.57,6.49,3.92
1,1,1.10,Premium,H,VS2,62.5,58.0,5729,6.59,6.54,4.10
2,2,1.50,Good,E,SI2,61.5,65.0,6300,7.21,7.17,4.42
3,3,1.53,Premium,E,SI1,61.3,59.0,12968,7.40,7.35,4.52
4,4,0.84,Fair,D,SI2,64.5,60.0,2167,5.92,5.84,3.79


In [5]:
train["cut"] = train["cut"].map({"Ideal": 5, 
                               "Premium": 4, 
                               "Very Good": 3,
                               "Good": 2,
                               "Fair": 1})
test["cut"] = test["cut"].map({"Ideal": 5, 
                               "Premium": 4, 
                               "Very Good": 3,
                               "Good": 2,
                               "Fair": 1})
train['train']  = 1
test['train']  = 0
together = pd.concat([train, test], axis=0,sort=False)
together["color"] = together["color"].map({'G':4,'E':6,'F':5,'H':3,'D':7,'I':2,'J' : 1})
together['clarity']=together['clarity'].map({'SI1' : 3, 'VS2' : 4, 'SI2': 2,'VS1' : 5,'VVS2' : 6, 'VVS1': 7, 'IF' : 8,'I1' : 1})

In [6]:
together

,id,carat,cut,color,clarity,depth,table,price,x,y,z,train
0,0,1.01,3,6,2,60.0,60.0,4540.0,6.57,6.49,3.92,1
1,1,1.10,4,3,4,62.5,58.0,5729.0,6.59,6.54,4.10,1
2,2,1.50,2,6,2,61.5,65.0,6300.0,7.21,7.17,4.42,1
3,3,1.53,4,6,3,61.3,59.0,12968.0,7.40,7.35,4.52,1
4,4,0.84,1,7,2,64.5,60.0,2167.0,5.92,5.84,3.79,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10784,10784,0.53,5,5,4,62.3,55.0,NaN,5.20,5.17,3.23,0
10785,10785,0.41,4,4,3,61.3,60.0,NaN,4.75,4.81,2.93,0
10786,10786,0.51,4,4,4,62.4,58.0,NaN,5.11,5.12,3.19,0
10787,10787,1.08,2,1,2,63.2,59.0,NaN,6.40,6.57,4.10,0


In [7]:
together["size"] = together["x"] * together["y"] * together["z"]
print(together)

          id  carat  cut  color  clarity  depth  table    price     x     y  \
0          0   1.01    3      6        2   60.0   60.0   4540.0  6.57  6.49   
1          1   1.10    4      3        4   62.5   58.0   5729.0  6.59  6.54   
2          2   1.50    2      6        2   61.5   65.0   6300.0  7.21  7.17   
3          3   1.53    4      6        3   61.3   59.0  12968.0  7.40  7.35   
4          4   0.84    1      7        2   64.5   60.0   2167.0  5.92  5.84   
...      ...    ...  ...    ...      ...    ...    ...      ...   ...   ...   
10784  10784   0.53    5      5        4   62.3   55.0      NaN  5.20  5.17   
10785  10785   0.41    4      4        3   61.3   60.0      NaN  4.75  4.81   
10786  10786   0.51    4      4        4   62.4   58.0      NaN  5.11  5.12   
10787  10787   1.08    2      1        2   63.2   59.0      NaN  6.40  6.57   
10788  10788   0.30    5      6        6   60.9   57.0      NaN  4.32  4.35   

          z  train        size  
0      3.92      1

In [8]:
object_columns_together = together.select_dtypes(include=['object'])
numerical_columns_together =together.select_dtypes(exclude=['object'])
restobject_columns = object_columns_together.select_dtypes(include=['object'])
together = pd.get_dummies(together, columns=restobject_columns.columns) 

In [9]:
final = together

In [10]:
tog_train = final[final['train'] == 1]
tog_train = tog_train.drop(['train',],axis=1)


tog_test = final[final['train'] == 0]
tog_test = tog_test.drop(['price'],axis=1)
tog_test = tog_test.drop(['train',],axis=1)

In [11]:
testtarget= tog_train['price']
tog_train = tog_train.drop(['price'],axis=1)

In [12]:
testtarget

0         4540.0
1         5729.0
2         6300.0
3        12968.0
4         2167.0
          ...   
43149     4975.0
43150     1617.0
43151     1014.0
43152     2871.0
43153     6320.0
Name: price, Length: 43154, dtype: float64

In [13]:
x_train,x_test,y_train,y_test = train_test_split(tog_train,testtarget,test_size=0.2,random_state=0)

In [14]:
xgb =XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=4, min_child_weight=1.5, n_estimators=2400,
             n_jobs=1, nthread=None, objective='reg:linear',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)


lgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=12000, 
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.4, 
                                       )

In [15]:
xgb.fit(x_train, y_train)
lgbm.fit(x_train, y_train,eval_metric='rmse')

[01:33:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

In [16]:
predict1 = xgb.predict(x_test)
predict = lgbm.predict(x_test)

In [17]:
xgb.fit(tog_train, testtarget)
lgbm.fit(tog_train, testtarget,eval_metric='rmse')

[01:34:42] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

In [18]:
predict4 = lgbm.predict(tog_test)
predict3 = xgb.predict(tog_test)
predict_y = ( predict3*0.3 + predict4 * 0.7)

In [19]:
submission = pd.DataFrame({
        "id": test["id"],
        "price": predict_y
    })
submission.to_csv('submission.csv', index=False)